In [2]:
import pandas as pd 
from gensim.models import Word2Vec
from nltk.corpus import stopwords 
import numpy as np
from nltk.probability import FreqDist, MLEProbDist
import scipy
from scipy.linalg import svd
from sklearn.decomposition import PCA

In [92]:
def pre(filename):
    stop_words = set(stopwords.words('english')) 
    maharashtra = pd.read_csv(filename,index_col = False).sort_values('questions')

    query = list(maharashtra['questions'])
    kccans = list(maharashtra['answers'])
    district = list(maharashtra['district'])
    state = list(maharashtra['state'])

    final = []
    for i,w in enumerate(query):
        if query[i] != kccans[i]:
            if type(query[i]) == str:
                query[i] = query[i].replace('?','')
                query[i] = query[i].replace('(','')
                query[i] = query[i].replace(')','')

        if type(query[i]) != float:
            if type(kccans[i][0]) != float:
                row = [query[i].lower().strip(),kccans[i].lower().strip(),district[i].lower().strip(),state[i].lower().strip()]
                if row[0].isalpha() or len(row[0])>5:
                    final.append(row)

    maharashtra = pd.DataFrame(final)
    maharashtra.columns = ['Query','Ans','District','State']

    main = []
    for w in list(maharashtra['Query']):
        main.append(w.split(' '))

    new_maharashtra = []
    all_words = []
    for w,i in enumerate(main):
        temp = []
        for j in i:
            if j not in stop_words:
                temp.append(j)
                all_words.append(j)

        #new_maharashtra.append([maharashtra['District'][w],maharashtra["State"][w]] + temp)
        new_maharashtra.append(temp)

    t = FreqDist(all_words)
    u = FreqDist(district)
    v = FreqDist(state)
    
    return u,v,t,new_maharashtra,maharashtra

In [88]:
def word2vec_QAmodel(u,v,t,new_maharashtra,dimen,a,**pca):
        
    for key, value in pca.items(): 
        key = value

    model = Word2Vec(new_maharashtra, min_count = 1,size=dimen)

    word2vec_value = []
    for i in new_maharashtra:
        value = np.array([0.0 for k in range(dimen)])
        count = len(i)
        c = 0
        for j in i:
            c += 1
#             if c in [1]:
#                 factor = 1/(1 + u[j]/u.N())
#             if c in [2]:
#                 factor = 0.0001/(0.0001 + v[j]/v.N())
#             if c in [3,4,5]:
#                 factor = 1#a/(a + t[j]/t.N())
            factor = 1
            value += model[j]*factor

        value = value/count
        word2vec_value.append(value)
    
    if pca == 'Yes':
        X = np.array(word2vec_value)
        pca = PCA()
        pca.fit(X)

        u = pca.components_[0]
        u_t = np.transpose(u)
        u_ut = np.matmul(u,u_t)

        new_word2vec_value = []
        for i,w in enumerate(word2vec_value):
            sub = w - u_ut*np.array(w)
            new_word2vec_value.append(sub)

        word2vec_value = new_word2vec_value

    model.save('model_word2vec.bin')
    word2vec_value = np.matrix(word2vec_value)
    
    np.save('word2vec_value',word2vec_value)

In [89]:
def test_query(u,v,t,input_list,dimen,no_similar,a,**pca):
    
    word2vec_value = np.load('word2vec_value.npy')
    model = Word2Vec.load('model_word2vec.bin')
    
    district = input_list[0]
    state = input_list[1]
    sent = input_list[2]

    #sent_words = [district,state] + sent.split(" ")
    sent_words = sent.split(" ")

    sent_value = np.array([0.0 for k in range(dimen)])

    sent_new = []
    count = 0

    for i in sent_words:
        if i not in stop_words:
            try:
#                 if count in [0]:
#                     factor = 1/(1 + u[i]/u.N())
#                 if count in [1]:
#                     factor = 0.0001/(0.0001 + v[i]/v.N())
#                 if count in [2,3,4]:
#                     factor = 1#a/(a + t[i]/t.N())
                factor = 1
                sent_value += model[i]*factor
                count += 1

            except:
                count += 1
                continue

    sent_value = sent_value/count
    
    if pca == 'Yes':
        sent_value = sent_value - u_ut*np.array(sent_value)

    all_dist = []
    for i in word2vec_value:
        dist = scipy.spatial.distance.cosine(i,sent_value)
        all_dist.append(dist)

    k = no_similar
    ind = np.argpartition(all_dist, k)[:k]
    
    return ind

In [90]:
def print_ans(ind, pdf, k):
    
    #pdf = maharashtra
    pdf = pdf.reset_index()
    
    print 'Top-%d\n\n'%(k)
    for i in ind:
        print 'Question: %s\nAnswer: %s\n\n'%(pdf['Query'][i],pdf['Ans'][i])
    

In [91]:
# u,v,t,new_maharashtra,maharashtra = preproc('all_files.csv')

# word2vec_QAmodel(u,v,t,new_maharashtra,50,0.001,pca='Yes')

# ind = test_query(u,v,t,['pune','maharashtra','blight attack on paddy'],50,5,0.001,pca='Yes')

# print_ans(ind, maharashtra, 5)

In [4]:
main = pd.DataFrame()
for i in range(1,36):
    t = pd.read_csv('11_%d.csv'%(i))
    main = pd.concat([main,t])

In [5]:
main

,Unnamed: 0,BlockName,Category,CreatedOn,Crop,DistrictName,KCCAns,QueryText,QueryType,Season,Sector,StateName
0,0,ANDHERI ...,Others,2017-01-08 11:15:46.120,Others,Mumbai Suburban,Your SMS Registration process successfully com...,Asked About SMS Activation?,Government Schemes,RABI,AGRICULTURE,MAHARASHTRA
1,1,ANDHERI ...,Others,2017-01-11 15:57:00.280,Others,Mumbai Suburban,recommended for Aside from DIB hole clean fro...,farmer want to know information about collecti...,Cultural Practices\t,RABI,AGRICULTURE,MAHARASHTRA
2,2,ANDHERI ...,Fruits,2017-01-24 15:22:42.027,Pomegranate,Mumbai Suburban,"Jyothi, Ganesh, Co 1, YCD 1, Araktha, Rudhra, ...",Farmer wants to know information about varieti...,Cultural Practices,RABI,HORTICULTURE,MAHARASHTRA
3,3,BORIVALI ...,Fruits,2017-02-14 11:46:43.173,Orange,Mumbai Suburban,FERTILIZER FOR ORANGE = 0:52;34 60 GM + BOROC...,FERTILIZER FOR ORANGE,Fertilizer Use and Availability,KHARIF,HORTICULTURE,MAHARASHTRA
4,4,ANDHERI ...,Others,2017-02-26 14:07:29.170,Others,Mumbai Suburban,Recommended for Aside from DIB hole clean from...,Farmer needs information regarding collection ...,Cultural Practices\t,RABI,AGRICULTURE,MAHARASHTRA
5,5,BORIVALI ...,Others,2017-06-25 10:45:52.367,Others,Mumbai Suburban,NATIONAL INSURANCE COMPANY LTD. - 1800 200 7710,ASKING ABOUT CONTACT NUMBER FOR PRADHANMANTRI ...,Crop Insurance,KHARIF,AGRICULTURE,MAHARASHTRA
0,0,SHAHAPUR ...,Vegetables,2017-01-01 13:03:47.020,Cucumber,THANE,Spray super confidor 10 ml /15 liter of water(...,Attack of Aphids,\tPlant Protection\t,RABI,HORTICULTURE,MAHARASHTRA
1,1,VIKRAMGAD ...,Others,2017-01-01 13:35:02.270,Others,THANE,?????? ?????? (??./??.)-?.? - ?\r\n???????? ??...,?????? ???-?????(???-??? ? ???-????),\tPlant Protection\t,RABI,AGRICULTURE,MAHARASHTRA
2,2,BHIWANDI,Vegetables,2017-01-02 10:34:11.430,Cucumber,THANE,spray m-45 30 gm in 15 liter water.,Attack of Blight?,\tPlant Protection\t,RABI,HORTICULTURE,MAHARASHTRA
3,3,BHIWANDI,Others,2017-01-02 16:12:09.150,Others,THANE,This week weather is cloudy & not Possibility ...,farmer asked about Weather Report ?,Weather,RABI,AGRICULTURE,MAHARASHTRA


In [6]:
pd.read_csv('all_files.csv')

,Unnamed: 0,answers,district,questions,state
0,0,['APPLICATION OF 10:26:26 2 BAG + UREA 20 KG +...,HINGOLI,NaN,MAHARASHTRA
1,1,['Krishi Vigyan Kendra PO: KanchanpurTal: Mira...,SANGLI,farmer asked about contact number of kvk sangali,MAHARASHTRA
2,2,['Website of patanjali ayurved =\r\ncustomerca...,PUNE,farmer asked about website of patanjali ayurved,MAHARASHTRA
3,3,['Goal 20 ml (oxyfluorfen)/15 liter of water-(...,NASIK,asking about weed management ground nut,MAHARASHTRA
4,4,['The rate of subsidy will be 25% of the capit...,NAGPUR,farmer asked about warehouse subsidy,MAHARASHTRA
5,5,['GROWTH SPRAY FOR GREEN GRAM SPRAY 12 61 00 ...,AHMADNAGAR,asked about growth spray for green gram,MAHARASHTRA
6,6,['Spray Admire 7 gm/15 Liter of Water (Imida 7...,BEED,attack of aphids and jassids on brinjal,MAHARASHTRA
7,7,['SPRAY MICROLA 45 ML + SAAF 40 GM/15 LITER WA...,AHMADNAGAR,yellow,MAHARASHTRA
8,8,['Spray Boom flower 30ml(DeviCropscience) + 12...,JALNA,for bloosming on papaya,MAHARASHTRA
9,9,['GROUND NUT PODS (DRY)-2800-4200/QTL.'],WARDHA,market rate of ground nut pods dry,MAHARASHTRA
